In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import collections
import random
import sys
from operator import itemgetter

In [2]:
import os

path = os.path.join("Data","en-ud-train.conllu")

def read(path):
    '''Constructs a list(corpus) of lists(sentences) of dicts(relevant information per word)'''
    with open(path, 'r', encoding = 'utf8') as f:
        
        data =[]
        
        # Needed to select sentences
        first_sent = True  
        previous_line_hashtag = False
        
        #linenumber for debugging
        linenumber = 0
        
        for line in f:
            
            #empty linedict
            linedict = {}
            linenumber +=1
            
            # Skip newdoc id and skip sentence id. Skip whitelines
            if line[0] in ['#','r\n','\n']:
                if line[0] in ['#']:
                    previous_line_hashtag = True                    
                continue
                
            
            # empty list for sentence
            if (line[0] == '1' and previous_line_hashtag == True):
                previous_line_hashtag = False
                if first_sent == True:
                    sentence = []
                    first_sent = False
                else:
                    data.append(sentence)
                    sentence = []
            
            # strip tabs
            parts = line.strip().split("\t") 
            
            # Make sure that all lines have the same structure 
            if len(parts) != 10:
                print(linenumber)
                print(parts)
                print('not same structure')
                break
            
            # fill linedict
            for i, part in enumerate(parts):
                
                # only save the relevant parts
                if i not in [0,1,3,6,7]:
                    continue
                linedict[i] = part
            
            sentence.append(linedict)

        # Check if it parsed all lines
        print('lines parsed: ',linenumber, 'of the 242774')
        return data

eng_train = read(path)
print('Number of sentences:', len(eng_train),'\n')
print('Example of datastructure: \n\n', eng_train[0:2])

lines parsed:  242774 of the 242774
Number of sentences: 12542 

Example of datastructure: 

 [[{0: '1', 1: 'Al', 3: 'PROPN', 6: '0', 7: 'root'}, {0: '2', 1: '-', 3: 'PUNCT', 6: '1', 7: 'punct'}, {0: '3', 1: 'Zaman', 3: 'PROPN', 6: '1', 7: 'flat'}, {0: '4', 1: ':', 3: 'PUNCT', 6: '1', 7: 'punct'}, {0: '5', 1: 'American', 3: 'ADJ', 6: '6', 7: 'amod'}, {0: '6', 1: 'forces', 3: 'NOUN', 6: '7', 7: 'nsubj'}, {0: '7', 1: 'killed', 3: 'VERB', 6: '1', 7: 'parataxis'}, {0: '8', 1: 'Shaikh', 3: 'PROPN', 6: '7', 7: 'obj'}, {0: '9', 1: 'Abdullah', 3: 'PROPN', 6: '8', 7: 'flat'}, {0: '10', 1: 'al', 3: 'PROPN', 6: '8', 7: 'flat'}, {0: '11', 1: '-', 3: 'PUNCT', 6: '8', 7: 'punct'}, {0: '12', 1: 'Ani', 3: 'PROPN', 6: '8', 7: 'flat'}, {0: '13', 1: ',', 3: 'PUNCT', 6: '8', 7: 'punct'}, {0: '14', 1: 'the', 3: 'DET', 6: '15', 7: 'det'}, {0: '15', 1: 'preacher', 3: 'NOUN', 6: '8', 7: 'appos'}, {0: '16', 1: 'at', 3: 'ADP', 6: '18', 7: 'case'}, {0: '17', 1: 'the', 3: 'DET', 6: '18', 7: 'det'}, {0: '18', 1: '

In [3]:
# Construct sentences from the training data, default (for development purposes) is 1 sentence
def construct_sentences(corpus, n=1):
    '''Constructs a list of dicts(sentences) with words and indices of the words (and the <root> symbol)'''
    
    sentences = []
    n=3

    
    for sent in corpus[0:n]:
        # create mappings from index to words for faster computability
        i2words = {}
        
        for i, word in enumerate(sent):
#             Append 'root word'
            if word[0]=='1':
                i2words[i] = '<root>'
                
            i2words[i+1] = word[1]
            
        sentences.append(i2words)
    return sentences
        
        
sentences = construct_sentences(eng_train)
print(sentences)
    

[{0: '<root>', 1: 'Al', 2: '-', 3: 'Zaman', 4: ':', 5: 'American', 6: 'forces', 7: 'killed', 8: 'Shaikh', 9: 'Abdullah', 10: 'al', 11: '-', 12: 'Ani', 13: ',', 14: 'the', 15: 'preacher', 16: 'at', 17: 'the', 18: 'mosque', 19: 'in', 20: 'the', 21: 'town', 22: 'of', 23: 'Qaim', 24: ',', 25: 'near', 26: 'the', 27: 'Syrian', 28: 'border', 29: '.'}, {0: '<root>', 1: '[', 2: 'This', 3: 'killing', 4: 'of', 5: 'a', 6: 'respected', 7: 'cleric', 8: 'will', 9: 'be', 10: 'causing', 11: 'us', 12: 'trouble', 13: 'for', 14: 'years', 15: 'to', 16: 'come', 17: '.', 18: ']'}, {0: '<root>', 1: 'DPA', 2: ':', 3: 'Iraqi', 4: 'authorities', 5: 'announced', 6: 'that', 7: 'they', 8: 'had', 9: 'busted', 10: 'up', 11: '3', 12: 'terrorist', 13: 'cells', 14: 'operating', 15: 'in', 16: 'Baghdad', 17: '.'}]


In [4]:

def construct_graph(sentences, n=1):
    '''Constructs a list of dicts(sentences) where each sentence is represented as a graph,
    inspired by the Python documentation recommended structure (https://www.python.org/doc/essays/graphs/). 
    
    graphs = [graph_of_sentence_1, graph_of_sentnce_2,...]
    
    Where each graph has the structure:
        
    graph_i = {to (index of a word within a sentence): [(from, weight), (from, weight)],
             2: [(1, 0.5323), (3, 0.3452345),....,(n, weight)],
             ...}
    
             
    '''
    A = np.array([[-0.11975589, -5.21741343, -8.6723423,  -2.5274601,  -3.59689045],
              [-1.82375038, -2.3231895,  -3.83282852, -0.80953324, -1.29496992],
              [-1.99295557, -2.15490437, -3.59260106, -0.8520692,  -1.22507524],
              [-1.67292762, -2.1708827,  -3.46629667, -0.92334598, -1.31005192],
              [-1.8787086,  -2.23452401, -3.59553671, -0.8395828,  -1.26999497]])
    
    graphs = []
    graph = {}
    n=2
    
    # Create (fully conected) graph 
    # TODO: (with random weights at the moment, should change that to the weights given by the Neural Net)
    for sent in sentences[0:n]:
        
        length = len(sent)  
        for to_index, word in sent.items():
            
            # Make sure there is nog connection from a node to itself
            #TODO: Volgens mij hoeven we maar 1 keer de probabilities van elke arc er in te stoppen (dit wordt niet geupdate)
            # Daarom gebruik ik hieronder tuples, maar anders moeten we misschien iets anders gebruiken
            from_index = [i for i in range(length) if i != to_index]
            graph[to_index] = [(i, random.random()) for i in from_index]
            
        graphs.append(graph)
        
    return graphs
             
graphs = construct_graph(sentences)
print(graphs[:1])

[{0: [(1, 0.1641292079228548), (2, 0.08068755886936985), (3, 0.8850878142814157), (4, 0.7200808858884468), (5, 0.8262079834756237), (6, 0.526740267447558), (7, 0.6170134057832986), (8, 0.8831225299467588), (9, 0.5715394911415144), (10, 0.5840562738734276), (11, 0.46357257959943554), (12, 0.9270639146791023), (13, 0.7680430481971677), (14, 0.014516447157187029), (15, 0.5345447963937762), (16, 0.5479921945974191), (17, 0.40424642263482824), (18, 0.28440357974356034)], 1: [(0, 0.7650311573926171), (2, 0.008813621932078353), (3, 0.30083426399071733), (4, 0.3060927643677962), (5, 0.8578857624605376), (6, 0.1266231225109249), (7, 0.028498743570644525), (8, 0.8615570661944377), (9, 0.6236445413843283), (10, 0.5424294287427934), (11, 0.01969285318353231), (12, 0.3412909194689884), (13, 0.28446708616591376), (14, 0.8610217950335451), (15, 0.584807600038836), (16, 0.14749014100885238), (17, 0.6194810121845495), (18, 0.9578060789350542)], 2: [(0, 0.8963837902629574), (1, 0.5742091755204207), (3, 

In [13]:

def max_incomming(sentence):
    mst = {}
    # find maximum incomming arcs
    for word_index, incomming_arcs in sentence.items():
        # Itemgetter = faster than lambda method, see: 
        # https://stackoverflow.com/questions/30534377/finding-maximum-value-in-python-list-of-tuples
        maximum_incomming = max(incomming_arcs, key = itemgetter(1))[0]
        mst[word_index] = maximum_incomming
    return mst
                
            

def dfs(graph, start, end):
    """Depth-first search for cycles 
    https://stackoverflow.com/questions/40833612/find-all-cycles-in-a-graph-implementation
    
    """
    fringe = [(start, [])]
    while fringe:
        print('1')
        print(fringe)
        state, path = fringe.pop()
        if path and state == end:
            yield path
            continue
        for next_state in graph[state]:
            if next_state in path:
                continue
            fringe.append((next_state, path+[next_state]))           
            
def get_cycle(graph, start):
    sentence_length = len(graph)
    cycle = []
    curr_state = start
    counter = 0
#     next_state = graph[curr_state]

    for i in range(sentence_length):

        # append current_state to cycle 
        cycle.append(curr_state)

        # Set current 
        next_state = graph[curr_state]
        curr_state = graph[next_state]
#         print('graph[start]',graph[start])
#         print('graph[curr_state]',graph[next_state])
        if graph[start]==graph[curr_state]:
            cycle.append(next_state)
            return cycle

#         print('curr_state', curr_state,'\n next_state', next_state)

        
        # If it iterated over all words and did not encounter a cycle, there is no cycle starting from this word
        counter+=1
        if counter == sentence_length:
            print('found no cycle with startkey:', start, cycle)
            return 
        

        
    print('out of while')



def MST(graphs, n=1):
    # n=1 for development purposes
    '''Takes a list of graphs and returns the Maximum Spanning Tree per graph by using the Chu Liu Edmonds Algorithm'''
    
    msts = []

    
    for sentence in graphs[0:n]:
        
        mst = max_incomming(sentence)
        # find all cycles

        cycles = []
        print(mst)
        for key, value in mst.items():
            print('startkey',key)
            #Can only be one cycle per starting key (since they only have one incomming and outgoing edge)
            cycle = get_cycle(mst, key)
            cycles.append(cycle)



        # as long as there are cycles, create a new graph for the sentence without the nodes
        # that formed a cycle, but with the new colapsed node
        while cycles:
            print(mst)
            print(cycles)
            return

            print('cycle TODO')
        
        # If no cycles, append mst to msts
        if not cycles:
            msts.append(mst)
        
        # If cycles, collapse cycles and find maximum incomming arcs
        #Recursive???


        # append mst to msts
#         msts.append(mst)
    return msts
        
mst_list = MST(graphs)
# print(mst_list[:1])





{0: 12, 1: 18, 2: 0, 3: 10, 4: 11, 5: 0, 6: 13, 7: 13, 8: 14, 9: 5, 10: 3, 11: 4, 12: 0, 13: 2, 14: 13, 15: 5, 16: 8, 17: 5, 18: 13, 19: 29, 20: 28, 21: 14, 22: 25, 23: 8, 24: 19, 25: 1, 26: 15, 27: 28, 28: 7, 29: 9}
startkey 0
startkey 1
found no cycle with startkey: 1 [1, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
startkey 2
startkey 3
startkey 4
startkey 5
startkey 6
found no cycle with startkey: 6 [6, 2, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
startkey 7
found no cycle with startkey: 7 [7, 2, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
startkey 8
found no cycle with startkey: 8 [8, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
startkey 9
found no cycle with startkey: 9 [9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
